<a href="https://colab.research.google.com/github/srimaailuri/Datascience/blob/main/transfer_learning_using_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import tensorflow as tf
import numpy as np
from google.colab import drive
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
plt.rcParams['figure.figsize']= (20,8)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/srima_DL_Lab/transfer_learning'

/content/gdrive/My Drive/srima_DL_Lab/transfer_learning


In [ ]:
from keras.callbacks import *
filepath = "/content/gdrive/My Drive/srima_DL_Lab/transfer_learning/mycnn/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
!unzip animals.zip

Streaming output truncated to the last 5000 lines.
  inflating: animals/animals/dogs/dogs_00007.jpg  
  inflating: animals/animals/dogs/dogs_00008.jpg  
  inflating: animals/animals/dogs/dogs_00009.jpg  
  inflating: animals/animals/dogs/dogs_00010.jpg  
  inflating: animals/animals/dogs/dogs_00011.jpg  
  inflating: animals/animals/dogs/dogs_00012.jpg  
  inflating: animals/animals/dogs/dogs_00013.jpg  
  inflating: animals/animals/dogs/dogs_00014.jpg  
  inflating: animals/animals/dogs/dogs_00015.jpg  
  inflating: animals/animals/dogs/dogs_00016.jpg  
  inflating: animals/animals/dogs/dogs_00017.jpg  
  inflating: animals/animals/dogs/dogs_00018.jpg  
  inflating: animals/animals/dogs/dogs_00019.jpg  
  inflating: animals/animals/dogs/dogs_00020.jpg  
  inflating: animals/animals/dogs/dogs_00021.jpg  
  inflating: animals/animals/dogs/dogs_00022.jpg  
  inflating: animals/animals/dogs/dogs_00023.jpg  
  inflating: animals/animals/dogs/dogs_00024.jpg  
  inflating: animals/animals/do

In [ ]:
!pip install translate

In [ ]:
from translate import translate
translate


<module 'translate.translate' from '/usr/local/lib/python3.10/dist-packages/translate/translate.py'>

In [ ]:

for i in os.listdir('animals/animals'):
  print(i, len(os.listdir('animals/animals/' + i)))

cats 1000
dogs 1000
panda 1000


In [ ]:
#os.mkdir('train')
#os.mkdir('test')


#for i in os.listdir('animals/animals'):
 # os.mkdir('train/' + i)
 # os.mkdir('test/' + i)
for j in os.listdir('animals/animals/'+i)[:1000]:
  os.rename('animals/animals/'+i+'/'+j, 'train/'+i+'/'+j)
for j in os.listdir('animals/animals/'+i)[:400]:
  os.rename('animals/animals/'+i+'/'+j, 'test/'+i+'/'+j)

In [ ]:

def img_data(dir_path, target_size, batch, class_lst, pre_processs):
  if pre_processs:
    generate_object = ImageDataGenerator(preprocessing_function=pre_processs)
  else:
    generate_object = ImageDataGenerator()

  return (generate_object.flow_from_directory(dir_path, target_size=target_size, batch_size=batch,  #using flow_from_directory because all of our data is saved in form of directories.
                                              class_mode='sparse', classes=class_lst, shuffle=True))

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [ ]:
train_data_gen = img_data('train', (224, 224), 500, os.listdir('train'), preprocess_input)
# mobilenet. preprocess_input will scale input pixels between -1 and 1. input_shape: Optional shape tuple,
# only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3)
# (with channels_last data format) or (3, 224, 224) (with channels_first data format).

valid_data_gen = img_data('test', (224, 224), 500, os.listdir('test'), preprocess_input)

Found 1000 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224, 224, 3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
)

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable=False

In [ ]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [ ]:
save_ck = callbacks.ModelCheckpoint('.mdl_wt.hdf5', save_best_only=True, monitor='val_loss', mode='min')

model.fit(train_data_gen, batch_size=500, validation_data=valid_data_gen, callbacks=[elst, save_ck], epochs=10)

Epoch 1/10
2/2 [==============================] - ETA: 0s - loss: 1.0607 - accuracy: 0.5190 

2/2 [==============================] - 73s 26s/step - loss: 1.0607 - accuracy: 0.5190
Epoch 2/10
2/2 [==============================] - ETA: 0s - loss: 4.0666e-06 - accuracy: 1.0000 

2/2 [==============================] - 49s 22s/step - loss: 4.0666e-06 - accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - ETA: 0s - loss: 5.2452e-09 - accuracy: 1.0000 

2/2 [==============================] - 49s 22s/step - loss: 5.2452e-09 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 48s 22s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

...

2/2 [==============================] - 49s 24s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 48s 22s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 49s 23s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 49s 24s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 65s 39s/step - loss: 0.0000e+00 - accuracy: 1.0000


In [ ]:
from keras import applications

In [ ]:
imported_model=tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
imported_model.trainable=False

In [ ]:
model_VGG16= tf.keras.models.Sequential()
model_VGG16.add(base_model)
model_VGG16.add(GlobalAvgPool2D())
model_VGG16.add(Dense(1024, activation='relu'))
model_VGG16.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [ ]:
save_ck = callbacks.ModelCheckpoint('.mdl_wt.hdf5', save_best_only=True, monitor='val_loss', mode='min')

model.fit(train_data_gen, batch_size=500, validation_data=valid_data_gen, callbacks=[elst, save_ck], epochs=5)

Epoch 1/5
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 71s 26s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/5
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 49s 22s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/5
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 49s 23s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/5
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 51s 22s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/5
2/2 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000 

2/2 [==============================] - 48s 24s/step - loss: 0.0000e+00 - accuracy: 1.0000
